In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter

from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma, FAISS

from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

# Create a chat model instance
llm = ChatOpenAI(
   temperature=0.1,
#    streaming=True,
#    callbacks=[StreamingStdOutCallbackHandler()],
)

loader = UnstructuredFileLoader("./files/묵향일부분.txt")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
    # length_function=len,
)

embeddings = OpenAIEmbeddings()

cache_dir = LocalFileStore("./cache/")
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings,cache_dir,)


docs = loader.load_and_split(text_splitter=splitter)

vector_store = FAISS.from_documents(docs, cached_embeddings)

retirever = vector_store.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        ("human", "{question}"),
    ]
)

chain = {"context": retirever, "question": RunnablePassthrough()} | prompt | llm

chain.invoke("유니콘을 문장으로 쓰는 나라가 어딘가요?")


TypeError: list indices must be integers or slices, not str

In [6]:
result = vector_store.similarity_search("유니콘을 문장으로 쓰는 나라가 어딘가요?")

result

[Document(page_content='었기에, 분위기가 좀 그렇기는 했지만 지미는 호기심을 참지 못하고 물었다.\n"유니콘을 문장으로 쓰는 나라가 어딘데요?"\n갑작스런 지미의 질문에 정신을 차린 파시르는 쓴웃음을 잠시 머금더니 내뱉듯 말했다.\n"레니아 근위 기사단의 문장이다. 저 강대한 크루마 제국의..."\n"레니아 근위 기사단이라구요? 그렇다면 방금 그 타이탄들은 근위 타이탄이란 말입니까?"\n놀라서 묻는 지미의 얼굴을 힐끗 쳐다 본 파시르는 다시 드래곤이 쓰러져 있는 곳으로 시\n선을 돌렸다.', metadata={'source': './files/묵향일부분.txt'}),
 Document(page_content='었기에, 분위기가 좀 그렇기는 했지만 지미는 호기심을 참지 못하고 물었다.\n"유니콘을 문장으로 쓰는 나라가 어딘데요?"\n갑작스런 지미의 질문에 정신을 차린 파시르는 쓴웃음을 잠시 머금더니 내뱉듯 말했다.\n"레니아 근위 기사단의 문장이다. 저 강대한 크루마 제국의..."\n"레니아 근위 기사단이라구요? 그렇다면 방금 그 타이탄들은 근위 타이탄이란 말입니까?"\n놀라서 묻는 지미의 얼굴을 힐끗 쳐다 본 파시르는 다시 드래곤이 쓰러져 있는 곳으로 시\n선을 돌렸다.', metadata={'source': './files/묵향일부분.txt'}),
 Document(page_content='절망 등 여러 가지 색채를 띠며 순간순간 변하고 있었다.\n"나는 옛날 론드바르 제국의 기사였다. 아직도 눈을 감으면 기억이 나지. 론드바르 최후의\n날. 화염이 충천하던 왕궁, 비명을 지르며 뛰어가던 시민들... 나에게 힘이 없다는 것이 그렇\n게도 원망스러웠던 적은 없었다. 시민들을 학살하고, 도시를 파괴하던 그 문장. 하얀 유니콘\n의 문장을 절대로 잊을 수 없어."\n파시르의 중얼거리는 말만으로는 도대체 유니콘을 문장으로 쓰는 제국이 어딘지 알 수 없', metadata={'source': './files/묵향일부분.txt'}),
 Documen